In [0]:
df = spark.read.csv('/Volumes/pyspark_project/source/source_data/trips/', header=True, inferSchema=True, sep=',')

In [0]:
display(df)

In [0]:
df.schema

In [0]:
schema_trips = df.schema
schema_trips

In [0]:
files_and_folders = dbutils.fs.ls("/Volumes/pyspark_project/source/source_data/")
folders = [item.name for item in files_and_folders if item.isDir()]
folders = [folder.rstrip('/') for folder in folders]
for folder in folders:
    print(folder)


# Spark Streaming

In [0]:
for folder in folders:
    df = spark.read.csv(f'/Volumes/pyspark_project/source/source_data/{folder}/', header=True, inferSchema=True, sep=',')

    schema = df.schema

    df = spark.readStream.format('csv')\
        .option("header",True)\
        .schema(schema)\
        .load(f"/Volumes/pyspark_project/source/source_data/{folder}/")

    df.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", f"/Volumes/pyspark_project/bronze/checkpoint/{folder}")\
    .trigger(once=True)\
    .toTable(f"pyspark_project.bronze.{folder}")
    